string 分割成单词成为tokenization

Splitting a string into a list of words is known as tokenization. One of the most popular tokenization comes from NLTK (Natural Language Tool Kit).

In [1]:
import os
os.environ['all_proxy'] = "socks5://127.0.0.1:10808"

In [2]:
import nltk
# nltk.set_proxy("socks5://127.0.0.1:10808")
# https://zhuanlan.zhihu.com/p/599003991 手动下载
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
from nltk.tokenize import word_tokenize
sentence = "hi, how are you?"
sentence.split()
word_tokenize(sentence)

['hi', ',', 'how', 'are', 'you', '?']

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
# create a corpus of sentences
corpus = [
    "hello, how are you?",
    "im getting bored at home. And you? What do you think?",
    "did you know about counts",
    "let's see if this works!",
    "YES!!!!"
]
# initialize CountVectorizer
ctv = CountVectorizer()
# fit the vectorizer on corpus
ctv.fit(corpus)
corpus_transformed = ctv.transform(corpus)

In [5]:
print(corpus_transformed)

  (0, 2)	1
  (0, 9)	1
  (0, 11)	1
  (0, 22)	1
  (1, 1)	1
  (1, 3)	1
  (1, 4)	1
  (1, 7)	1
  (1, 8)	1
  (1, 10)	1
  (1, 13)	1
  (1, 17)	1
  (1, 19)	1
  (1, 22)	2
  (2, 0)	1
  (2, 5)	1
  (2, 6)	1
  (2, 14)	1
  (2, 22)	1
  (3, 12)	1
  (3, 15)	1
  (3, 16)	1
  (3, 18)	1
  (3, 20)	1
  (4, 21)	1
